In [6]:
import pygame
import math
import colorsys
import cv2
import numpy as np
from IPython.display import clear_output

# Initialize Pygame
pygame.init()

# Further reduced resolution
width, height = 320, 320  # Reduced from 480x480
screen = pygame.Surface((width, height))

# Video writer with maximum compression
fourcc = cv2.VideoWriter_fourcc(*'avc1')
video = cv2.VideoWriter(
    'ColorfulSpiralPattern.mp4',
    fourcc,
    24.0,  # Further reduced from 30 fps to 24 fps
    (width, height),
    isColor=True
)

# Constants
PHI = (1 + math.sqrt(5)) / 2
GOLDEN_ANGLE = math.pi * 2 * (1 - 1 / PHI)

# Animation loop
t = 0
frames = []  # Store frames for two-pass encoding

while t < 3:  # Further reduced from 5 seconds to 3 seconds
    screen.fill((0, 0, 0))
    
    max_points = 200  # Further reduced from 300
    base_radius = min(width, height) / 4
    
    for i in range(max_points):
        distance = (i / max_points) * base_radius
        angle = i * GOLDEN_ANGLE + t
        
        wave = math.sin(i * 0.1 + t) * 10  # Further reduced wave amplitude
        x = math.cos(angle) * (distance + wave) + width/2
        y = math.sin(angle) * (distance + wave) + height/2
        
        size = 1 + math.sin(i * 0.1 + t)  # Minimized point size
        hue = (i * 2 + t * 50) % 360 / 360
        light = 0.5 + math.sin(i * 0.1 + t) * 0.2
        rgb = [int(x * 255) for x in colorsys.hls_to_rgb(hue, light, 1)]
        
        pygame.draw.circle(screen, rgb, (int(x), int(y)), int(size))
        
        # Further reduced connecting lines
        if i > 0 and i % 12 == 0:  # Changed from 8 to 12
            prev_dist = ((i - 12) / max_points) * base_radius
            prev_angle = (i - 12) * GOLDEN_ANGLE + t
            prev_x = math.cos(prev_angle) * prev_dist + width/2
            prev_y = math.sin(prev_angle) * prev_dist + height/2
            pygame.draw.line(screen, (*rgb, 77), (x, y), (prev_x, prev_y), 1)
    
    # Prepare frame with maximum compression
    pixels = pygame.surfarray.array3d(screen)
    pixels = pixels.swapaxes(0,1)
    pixels = cv2.cvtColor(pixels, cv2.COLOR_RGB2BGR)
    
    # Apply aggressive compression
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 70]  # Reduced quality from 85 to 70
    _, encoded = cv2.imencode('.jpg', pixels, encode_param)
    decoded = cv2.imdecode(encoded, 1)
    
    # Store frame for two-pass encoding
    frames.append(decoded)
    
    t += 0.05  # Further increased time step
    clear_output(wait=True)
    print(f"Rendering: {int(t*33)}%")

# Two-pass encoding for better compression
print("Performing two-pass encoding for better compression...")
temp_file = 'temp_video.mp4'
final_file = 'ColorfulSpiralPattern.mp4'

# First pass with lower bitrate
temp_video = cv2.VideoWriter(
    temp_file,
    fourcc,
    24.0,
    (width, height),
    isColor=True
)

for frame in frames:
    temp_video.write(frame)
temp_video.release()

# Second pass with even lower bitrate
cap = cv2.VideoCapture(temp_file)
final_video = cv2.VideoWriter(
    final_file,
    fourcc,
    24.0,
    (width, height),
    isColor=True
)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Additional compression on second pass
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 60]
    _, encoded = cv2.imencode('.jpg', frame, encode_param)
    decoded = cv2.imdecode(encoded, 1)
    final_video.write(decoded)

cap.release()
final_video.release()
pygame.quit()

# Clean up temporary file
import os
os.remove(temp_file)

print("Video saved as 'ColorfulSpiralPattern.mp4'")

Rendering: 100%
Performing two-pass encoding for better compression...


Video saved as 'ColorfulSpiralPattern.mp4'
